In [5]:
from dotenv import load_dotenv
import os
import requests
import json
import pandas as pd
import quandl
import matplotlib
import hvplot.pandas

%matplotlib inline

In [6]:
load_dotenv()

True

In [7]:
downtown_sf_zipcodes = pd.read_csv('downtown_SF_zipcodes.csv')
downtown_sf_zipcodes.head()

,region_id,region_type,region
0,97559,zip,94104; CA; San Francisco-Oakland-Hayward; San ...
1,97563,zip,94108
2,97566,zip,94111; CA; San Francisco-Oakland-Hayward; San ...
3,97587,zip,94133


In [8]:
outside_sf_zipcodes = pd.read_csv('outside_SF_zipcodes.csv')
outside_sf_zipcodes.head()

,region_id,region_type,region
0,97583,zip,94129; CA; San Francisco-Oakland-Hayward; San ...
1,97571,zip,94117
2,97565,zip,94110
3,97578,zip,94124; CA; San Francisco-Oakland-Hayward; San ...
4,97581,zip,94127; CA; San Francisco-Oakland-Hayward; San ...


In [9]:
quandl.ApiConfig.api_key = os.getenv('QUANDL_API_KEY')

In [10]:
num_of_codes = len(downtown_sf_zipcodes.index)
num_of_codes

4

In [11]:
# Pull downtown San Francisco data for single-family homes time series from zillow through Quandl API

data = {}
count = 0
while count < num_of_codes:
    zipcode_code = downtown_sf_zipcodes.iloc[count,0]
    data[count] = quandl.get_table('ZILLOW/DATA', indicator_id='ZSFH', region_id=zipcode_code)
    data[count] = data[count].set_index('date')
    count += 1
    
data[3].head()

,indicator_id,region_id,value
date,,,
2020-12-31,ZSFH,97587,2664073.0
2020-11-30,ZSFH,97587,2699389.0
2020-10-31,ZSFH,97587,2729402.0
2020-09-30,ZSFH,97587,2866089.0
2020-08-31,ZSFH,97587,2699777.0


In [12]:
# Combine downtown San Francisco data for single-family homes into a single dataframe

first_data = 'yes'
count = 0
dataframe_count = 0
downtown_sf_sfh = pd.DataFrame()

while count < num_of_codes:
    if len(data[count].index) == 0 and first_data == 'yes':
        count += 1
    elif len(data[count].index) > 0 and first_data == 'yes':
        dataframe_count = 1
        downtown_sf_sfh['indicator_id'] = data[count]['indicator_id']
        downtown_sf_sfh['value'] = data[count]['value']
        downtown_sf_sfh['count'] = dataframe_count
        first_data = 'no'
        count += 1
    elif len(data[count].index) == 0 and first_data == 'no':
        count += 1
    elif len(data[count].index) > 0 and first_data == 'no':
        dataframe_count += 1
        downtown_sf_sfh['value'] += data[count]['value']
        downtown_sf_sfh['count'] = dataframe_count
        count += 1

downtown_sf_sfh['mean'] = round(downtown_sf_sfh['value']/downtown_sf_sfh['count'],2)
downtown_sf_sfh.head()

,indicator_id,value,count,mean
date,,,,
2020-12-31,ZSFH,6647362.0,3,2215787.33
2020-11-30,ZSFH,6729250.0,3,2243083.33
2020-10-31,ZSFH,6784914.0,3,2261638.00
2020-09-30,ZSFH,7335536.0,3,2445178.67
2020-08-31,ZSFH,6738826.0,3,2246275.33


In [14]:
data[0].to_csv(r'sfh0.csv')
data[1].to_csv(r'sfh1.csv')
data[2].to_csv(r'sfh2.csv')
data[3].to_csv(r'sfh3.csv')